In [1]:
%run -i "../util/file_utils.ipynb"
%run -i "../util/util_simple_classifier.ipynb"

In [2]:
import re
from sklearn.metrics import classification_report
from openai import OpenAI
client = OpenAI(api_key=OPEN_AI_KEY)

In [3]:
train_dataset = load_dataset("SetFit/bbc-news", split="train")
test_dataset = load_dataset("SetFit/bbc-news", split="test")

Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.


In [4]:
example = test_dataset[0]["text"]
category = test_dataset[0]["label_text"]
print(example)
print(category)

carry on star patsy rowlands dies actress patsy rowlands  known to millions for her roles in the carry on films  has died at the age of 71.  rowlands starred in nine of the popular carry on films  alongside fellow regulars sid james  kenneth williams and barbara windsor. she also carved out a successful television career  appearing for many years in itv s well-loved comedy bless this house. rowlands died in hove on saturday morning  her agent said.  born in january 1934  rowlands won a scholarship to the guildhall school of speech and drama scholarship when she was just 15.  after spending several years at the players theatre in london  she made her film debut in 1963 in tom jones  directed by tony richardson. she made her first carry on film in 1969 where she appeared in carry on again doctor. rowlands played the hard-done-by wife or the put-upon employee as a regular carry on star. she also appeared in carry on at your convenience  carry on matron and carry on loving  as well as othe

In [5]:
# Run GPT-3 for one example
prompt="""You are classifying texts by topics. There are 5 topics: tech, entertainment, business, politics and sport. 
Output the topic and nothing else. For example, if the topic is business, your output should be "business".
Give the following text, what is its topic from the above list without any additional explanations: """ + example
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=256,
    top_p=1.0,
    frequency_penalty=0,
    presence_penalty=0,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ], 
)
print(response.choices[0].message.content)

entertainment


In [6]:
# Evaluate GPT classification on several examples
def get_gpt_classification(input_text):
    prompt="""You are classifying texts by topics. There are 5 topics: tech, entertainment, business, politics and sport. 
Output the topic and nothing else. For example, if the topic is business, your output should be "business".
Give the following text, what is its topic from the above list without any additional explanations: """ + input_text
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        max_tokens=256,
        top_p=1.0,
        frequency_penalty=0,
        presence_penalty=0,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ], 
    )
    classification = response.choices[0].message.content
    classification = classification.lower().strip()
    return classification

In [7]:
test_df = test_dataset.to_pandas()
test_df.sample(frac=1)
test_data = test_df[0:200].copy()

In [8]:
test_data["gpt_prediction"] = test_data["text"].apply(lambda x: get_gpt_classification(x))

In [9]:
def get_one_word_match(input_text):
    loc = re.search(r'tech|entertainment|business|sport|politics', input_text).span()
    return input_text[loc[0]:loc[1]]
test_data["gpt_prediction"] = test_data["gpt_prediction"].apply(lambda x: get_one_word_match(x))

In [10]:
label_list = ["tech", "business", "sport", "entertainment", "politics"]
test_data["gpt_label"] = test_data["gpt_prediction"].apply(lambda x: label_list.index(x))

In [11]:
print(test_data)

                                                  text  label     label_text  \
0    carry on star patsy rowlands dies actress pats...      3  entertainment   
1    sydney to host north v south game sydney will ...      2          sport   
2    uk coal plunges into deeper loss shares in uk ...      1       business   
3    blair joins school sailing trip the prime mini...      4       politics   
4    bath faced with tindall ultimatum mike tindall...      2          sport   
..                                                 ...    ...            ...   
195  french wine gets 70m euro top-up the french go...      1       business   
196  argonaut founder rebuilds empire jez san  the ...      0           tech   
197  half-life 2 sweeps bafta awards pc first perso...      0           tech   
198  xbox power cable  fire fear  microsoft has sai...      0           tech   
199  prop jones ready for hard graft adam jones say...      2          sport   

    gpt_prediction  gpt_label  
0    en

In [12]:
print(classification_report(test_data["label"], test_data["gpt_label"], target_names=label_list))

               precision    recall  f1-score   support

         tech       0.97      0.80      0.88        41
     business       0.87      0.89      0.88        44
        sport       1.00      0.96      0.98        48
entertainment       0.88      0.90      0.89        40
     politics       0.76      0.96      0.85        27

     accuracy                           0.90       200
    macro avg       0.90      0.90      0.90       200
 weighted avg       0.91      0.90      0.90       200

